---
layout: post
title:  "NLP: LSI (Latent Semantic Indexing) and LDA (Latent Dirichlet Allocation)"
date:   2023-04-13 10:14:54 +0700
categories: MachineLearning
---
# TOC

# Introduction

Turning words into vector is a good practice in natural language processing. This digitalization enables us to do matrix computation and statistical inference on corpus. One common way to do that is the tf-idf weighting scheme, in which we count the frequency of word appreance in each document (it could be another scheme), resulting in a matrix of row to be word and column to be document. This bag of word approach considers word occurance but doesn't care about the position of words. Still, we can safely assume that if two documents have similar words, they are about similar topic.

## Dot product

After reducing words and documents into vectors in the vector space, we can compute the similarity between words or query (words) and documents. This can be used for information retrieval, to retrieve the documents that are close to the query. How to calculate the similarity exactly? Since they are vector, we can calculate the cosine of the angle between them:

$$ sim(w_1, w_2) = \vec{v}(w_1) . \vec{v}(w_2) = \frac{\vec{V}(w_1).\vec{V}(w_2)}{\mid \vec{V}(w_1) \mid . \mid \vec{V}(w_2) \mid} $$

The numerator is the dot product between two vectors and the denominator is the product of their Euclidean lengths $$ \mid \vec{V}(w_1) \mid = \sqrt{\sum_{i=1}^{m} \vec{V}_{i}^{2}(w)} $$. The denominator is to length-normalize the vectors into the unit vector. 
Consider three documents SaS, PaP, WH to be the three novels: Sense and Sensibility by Jane Austen, Pride and Prejudice by Jane Austen, and Wuthering Heights by Emily Bronte. Consider three terms affection, jealous and gossip with their respective frequency in each of those novels.

|term | SaS | PaP | WH |
|--|--|--|--|
|affection|115|58|20|
|jealous|10|7|11|
|gossip|2|0|6|

The term frequencies are converted into:

|term | SaS | PaP | WH |
|--|--|--|--|
|affection|0.996|0.993|0.847|
|jealous|0.087|0.120|0.466|
|gossip|0.017|0|0.254|

The cosine similarities between SaS and PaP: $$ sim(\vec{v}(SaS), \vec{v}(PaP)) = 0.999 $$ whereas $$ sim(\vec{v}(SaS), \vec{v}(WH)) = 0.888 $$. Thus the two books by Austen are closer to each other than to the book by Bronte (when restricted to the three terms we consider, and when the frequency doesn't have idf multiplier).

If we have the query q = "jealous gossip", its unit vector $$ \vec{v}(q) = (0, 0.707, 0.707) (usually the number of dimensions equals the number of vocabulary M) $$, its similarity index with the three novels would be $$ \vec{v}(q) . \vec{v}(d) $$. With d = Wuthering Heights, the score is 0.509, for d = Price and Prejudge the score is 0.085, and for Sense and Sensibility the score is 0.074. This suggests the novel Wuthering Heights when you query for "jealousy gossip". 

## Singular value decomposition (SVD)

Before doing singular value decomposition, let's recap some linear algebra. Let C be an MxN matrix with real valued entries (for the term-document matrix with all entries non-negative). The rank of the matrix would be the number of linearly independent rows or columns and would be smaller than min M or N. A square matrix with all off diagonal entries to be zero is called a diagonal matrix, the rank equal to the number of non zero diagonal entries. If all the diagonal entries to be 1, it is called the identity matrix I.

For a square matrix MxM called C, we have a vector $$ \vec{x} $$ that is not all zeros and the value $$ \lambda $$ with $$ C \vec{x} = \lambda \vec{x} $$; $$ \lambda $$ being called the eigenvalues of C. The N-vector $$ \vec{x} $$ is called the right eigenvector. The eigenvector corresponding to the biggest eigenvalue woudl be called the principal eigenvector. The left eigenvector of C is the M-vector y such that: $$ \vec{y}^T C = \lambda \vec{y}^T $$. The number of non zero eigenvalues of C is at most rank(C).

We can solve for the eigenvalues of matrix by the characteristic equation: 

$$ (C - \lambda I_M) \vec{x} = 0 $$ which is equivalent to: 

$$ \mid(C - \lambda I_M)\mid = 0 $$ where $$ \mid S \mid $$ denotes the determinant of a square matrix S. This equation of an Mth order polynomial equation with at most M roots which are all eigenvalues of C. We would see that when we do matrix multiplication, the effect is determined by the eigen vectors and eigen values of it. Also, small eigenvalues would have small effect. We would also make use of the fact that, for a symmetric matrix S, the eigenvectors are orthogonal. So matrix decomposition is the ways in which a square matrix is factored into product of matrices derived from its eigenvectors. When we decompose non square matrix (like the term-document matrix), we have singular values, that can parallel the eigen values.

First, consider the theorem for the matrix diagonalization: Let S be a square real valued MxM matrix with M linearly independent eigenvectors. There exists an eigen decomposition: $$ S = U \Sigma U^{-1} $$ where the columns of U are the eigenvectors of S and $$ \Sigma $$ is a diagonal matrix whose diagonal entries are the eigenvalues of S in decreasing order. If the eigenvalues are distint, then this decomposition is unique. Then, for a symmetric matrix, there would be a symmetric diagonal decomposition where the columns of U are orthogonal and normalized eigenvectors of S.

Now we consider our singular value decomposition. Given a term-document matrix C (as above), it is not likely to be square or symmetric. So we consider $$ C.C^T $$ and its term matrix MxM U whose columns are the orthogonal eigenvectors of $$ C.C^T $$. Similarly, we consider its document matrix NxN V whose columns are the orthogonal eigenvectors of $$ C^T.C $$.

Let r be the rank of MxN matrix C, then there is a singular value decomposition of C in the form:

$$ C = U \Sigma V^T $$

$$ \Sigma $$ is the singular matrix with singular values in descending order in the diagonal. Then we can take k biggest singular values and compute the output $$ C_k = U \Sigma_k V^T $$.

This $$ C_k $$ would approximate C, and minimize the Frobenius norm of the matrix difference $$ X = C - C_k $$. The Frobenius distance is:

$$ \mid\mid X \mid\mid_{F} = \sqrt{\sum_{i=1}^{M} \sum_{j=1}^{N} X_{ij}^2} $$

Since $$ C_k $$ has rank k < r, we call $$ C_k $$ a low rank approximation. This is the construct of an approximate version of C, with less dimensions (this can be considered a dimensionality reduction technique). This sometimes gives better information (noise reduction) of the documents.

## Latent semantic indexing (LSI)

The above process is called LSI, when it uses SVD to find a lower rank matrix that can approximate the term document matrix C keeping critical information in the document. This yield a new representation for each document in the corpus and we can cast queries into this representation as well with the query-document similarity score. Furthermore, the LSI can relieve the challenges of mere word-count approach of synonymy and polysemy. Synonymy is the case that multiple words represent similar things, and polysemy is the case that one word can have different meaning. In word-count approach, since we treat each word differently, the computed similarity between a query (car) and a document containing both car and automobile would underestimate the true similarity that a user perceive. In the other case, when a word can mean different thing, a computed similarity basing on word count only would overestimate the true similarity that a user perceive.

In LSI, we can find a latent representation with far lower dimension, when a term document matrix with tens of thousands of rows and columns can be reduced to hundreds. Each row/column (term/document) in the matrix would be mapped to a k-dimensional space, defined by k principal eigenvectors (corresponding to the largest eigenvalues) of $$ C.C^T $$ and $$ C^T.C $$.

After having the new k-dimensional LSI representation, we can compute similarities between vectors. A query vector $$ \vec{q} $$ would be mapped into its LSI represenation by the transformation:

$$ \vec{q_k} = \Sigma_k^{-1} . U_k^T . \vec{q} $$

For example, consider the term-document matrix C

|term|d1|d2|d3|d4|d5|d6|
|--|--|--|--|--|--|--|
|ship|1|0|1|0|0|0|
|boat|0|1|0|0|0|0|
|ocean|1|1|0|0|0|0|
|voyage|1|0|0|1|1|0|
|trip|0|0|0|1|0|1|

The SVD is the product of the following three matrices:

U - the SVD term matrix:

|term|1|2|3|4|5|
|--|--|--|--|--|--|
|ship|-0.44|-0.30|0.57|0.58|0.25|
|boat|-0.13|-0.33|-0.59|0.00|0.73|
|ocean|-0.48|-0.51|-0.37|0.00|-0.61|
|voyage|-0.70|0.35|0.15|-0.58|0.16|
|trip|-0.26|0.65|-0.41|0.58|-0.09|

$$ \Sigma $$ - the SVD singular matrix:

|2.16|0.00|0.00|0.00|0.00|
|--|--|--|--|--|
|0.00|1.59|0.00|0.00|0.00|
|0.00|0.00|1.28|0.00|0.00|
|0.00|0.00|0.00|1.00|0.00|
|0.00|0.00|0.00|0.00|0.39|

$$ V^T $$ - the SVD document matrix:

|d|d1|d2|d3|d4|d5|d6|
|--|--|--|--|--|--|--|
|1|-0.75|-0.28|-0.20|-0.45|-0.33|-0.12|
|2|-0.29|-0.53|-0.19|0.63|0.22|0.41|
|3|0.28|-0.75|0.45|-0.20|0.12|-0.33|
|4|0.00|0.00|0.58|0.00|-0.58|0.58|
|5|-0.53|0.29|0.63|0.19|0.41|-0.22|